In [1]:
from neo4j.v1 import GraphDatabase
import pandas as pd
pd.set_option('display.max_rows', 10000)
pd.set_option('display.max_columns', 20)
pd.set_option('display.width', 100)
pd.set_option('display.max_colwidth', 15)
import re
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity ="all"
import numpy as np

In [2]:
df = pd.read_csv('https://storage.googleapis.com/finex_hot_wallet/july-sep-out')

In [3]:
#get set of to_addresses
to_address_set = set(df['to_address'])
#create df with one column as set of addresses
fdf = pd.DataFrame()
fdf['address'] = list(set(df['to_address']))
to_address_list = list(to_address_set)
len(to_address_list)

14046

In [4]:
sample_list = to_address_list[0:3999]

In [5]:
sample_list_df = pd.DataFrame()
sample_list_df['address'] = sample_list
len(sample_list_df)

3999

In [6]:
#tx_from_finex
value_count_df = df['to_address'].value_counts()
value_count_df = pd.DataFrame(value_count_df)
value_count_df = value_count_df.reset_index()
value_count_df.columns = ['address', 'from_finex_tx']
fdf = pd.merge(fdf, value_count_df, on ='address')
fdf = pd.merge(fdf, sample_list_df, on ='address')
len(fdf)

3999

In [7]:
#vol from finex
df['value'] = pd.to_numeric(df['value'], errors='coerce')
sum_value_df = df[['value', 'to_address']].groupby('to_address').sum()
sum_value_df = sum_value_df.reset_index()
sum_value_df.columns = ['address', 'from_finex_vol']
fdf = pd.merge(fdf, sum_value_df, on ='address')
len(fdf)

3999

In [8]:
#vol per tx from finex
fdf['vol_per_tx'] = (fdf['from_finex_vol'] / fdf['from_finex_tx'])
len(fdf)

3999

In [9]:
#is_smart_contract
smart_contract_tx_df = df.loc[df['input'] != '0x00']
smart_contract_tx_df = smart_contract_tx_df[['to_address', 'input']]
smart_contract_tx_df = smart_contract_tx_df.groupby('to_address').count()
smart_contract_tx_df = smart_contract_tx_df.reset_index()
smart_contract_tx_df.columns = ['address', 'smart_contract_num']
fdf = pd.merge(fdf, smart_contract_tx_df, on ='address', how='left')
fdf = fdf.fillna(0)
fdf['is_smart_contract'] = (fdf['smart_contract_num'] / fdf['from_finex_tx'])
fdf = fdf.drop(columns = ['smart_contract_num'])
len(fdf)

3999

In [10]:
#Bring 2nd_order_from_transactions
df = pd.read_csv('https://storage.googleapis.com/finex_hot_wallet/july_sep_2nd_degree_from_4k')
df.head()
len(df)
len(fdf)

,hash,nonce,transaction_index,from_address,to_address,value,gas,gas_price,input,receipt_cumulative_gas_used,receipt_gas_used,receipt_contract_address,receipt_root,receipt_status,block_timestamp,block_number,block_hash
0,0x652fa913f...,11224,11,0x909641069...,0x05f4a42e2...,0,200000,4000000000,0xa9059cbb0...,655891,22529,NaN,NaN,1,2018-11-24 ...,6766015,0xc06bc0c79...
1,0x6149c9de1...,3273,63,0xa8282a3d5...,0x5e3346444...,0,250000,9000000000,0xa9059cbb0...,4767809,36867,NaN,NaN,1,2018-11-24 ...,6765386,0xd2f352a8e...
2,0x694ddb2f0...,11218,87,0x909641069...,0x2ccbff3a0...,0,200000,5500000000,0xa9059cbb0...,6508434,38148,NaN,NaN,1,2018-11-24 ...,6764018,0x204cec255...
3,0xf0adfd0c5...,4932,76,0xd12cd8a37...,0xfa52274dd...,12591997931...,35000,6900000000,0xf7654176,7417968,30981,NaN,NaN,1,2018-11-24 ...,6766205,0xd80fca27a...
4,0x870e1e3d8...,4940,49,0xd12cd8a37...,0xfa52274dd...,45199967934...,35000,10350000000,0xf7654176,1662673,30981,NaN,NaN,1,2018-11-24 ...,6766500,0x725d6a774...


69722

3999

In [11]:
#nd_order_med_gas_from
df.head()
nd_order_med_gas_from_df = df[['from_address', 'gas']]
nd_order_med_gas_from_df = nd_order_med_gas_from_df.groupby('from_address').median()
nd_order_med_gas_from_df = nd_order_med_gas_from_df.reset_index()
nd_order_med_gas_from_df.columns = ['address', '2nd_order_med_gas_from']
fdf = pd.merge(fdf, nd_order_med_gas_from_df, on ='address', how='left')

,hash,nonce,transaction_index,from_address,to_address,value,gas,gas_price,input,receipt_cumulative_gas_used,receipt_gas_used,receipt_contract_address,receipt_root,receipt_status,block_timestamp,block_number,block_hash
0,0x652fa913f...,11224,11,0x909641069...,0x05f4a42e2...,0,200000,4000000000,0xa9059cbb0...,655891,22529,NaN,NaN,1,2018-11-24 ...,6766015,0xc06bc0c79...
1,0x6149c9de1...,3273,63,0xa8282a3d5...,0x5e3346444...,0,250000,9000000000,0xa9059cbb0...,4767809,36867,NaN,NaN,1,2018-11-24 ...,6765386,0xd2f352a8e...
2,0x694ddb2f0...,11218,87,0x909641069...,0x2ccbff3a0...,0,200000,5500000000,0xa9059cbb0...,6508434,38148,NaN,NaN,1,2018-11-24 ...,6764018,0x204cec255...
3,0xf0adfd0c5...,4932,76,0xd12cd8a37...,0xfa52274dd...,12591997931...,35000,6900000000,0xf7654176,7417968,30981,NaN,NaN,1,2018-11-24 ...,6766205,0xd80fca27a...
4,0x870e1e3d8...,4940,49,0xd12cd8a37...,0xfa52274dd...,45199967934...,35000,10350000000,0xf7654176,1662673,30981,NaN,NaN,1,2018-11-24 ...,6766500,0x725d6a774...


In [12]:
#2nd_order_tx_from
tx_count_df = df[['from_address', 'hash']].groupby('from_address').count()
len(tx_count_df)
tx_count_df = tx_count_df.reset_index()
tx_count_df.columns = ['address', '2nd_order_tx_from']
fdf = pd.merge(fdf, tx_count_df, on ='address', how='left')
len(fdf)

2508

3999

In [13]:
#2nd_order_vol_from
df['value'] = pd.to_numeric(df['value'], errors='coerce')
sum_value_df = df[['value', 'from_address']].groupby('from_address').sum()
sum_value_df = sum_value_df.reset_index()
sum_value_df.columns = ['address', '2nd_order_vol_from']
fdf = pd.merge(fdf, sum_value_df, on ='address', how='left')
fdf = fdf.fillna(0)

In [14]:
#2nd_order_avg_tx_val
fdf['2nd_order_vol_per_tx_from'] = (fdf['2nd_order_vol_from'] / fdf['2nd_order_tx_from'])
len(fdf)

3999

In [15]:
#smart_contract_%
smart_contract_tx_df = df.loc[df['input'] != '0x00']
smart_contract_tx_df = smart_contract_tx_df[['to_address', 'input']]
smart_contract_tx_df = smart_contract_tx_df.groupby('to_address').count()
smart_contract_tx_df = smart_contract_tx_df.reset_index()
smart_contract_tx_df.columns = ['address', 'smart_contract_num']
fdf = pd.merge(fdf, smart_contract_tx_df, on ='address', how='left')
fdf = fdf.fillna(0)
fdf['2nd_order_smart_contract_%'] = (fdf['smart_contract_num'] / fdf['2nd_order_tx_from'])
fdf = fdf.drop(columns = ['smart_contract_num'])
fdf['2nd_order_smart_contract_%'] = fdf['2nd_order_smart_contract_%'].replace([np.inf, -np.inf], 1)
len(fdf)

3999

In [16]:
#Bring 2nd_order_to_transactions
df = pd.read_csv('https://storage.googleapis.com/finex_hot_wallet/july_sep_2nd_degree_to_4k')
df.head()
len(df)
len(fdf)

,hash,nonce,transaction_index,from_address,to_address,value,gas,gas_price,input,receipt_cumulative_gas_used,receipt_gas_used,receipt_contract_address,receipt_root,receipt_status,block_timestamp,block_number,block_hash
0,0xd2ef77134...,673,35,0x29873bd15...,0xbf52f2ab3...,0,120000,21000000000,0xa9059cbb0...,2255311,52481,NaN,NaN,1,2018-12-03 ...,6820184,0x34104d641...
1,0xe216aa7f6...,671,25,0x29873bd15...,0xbf52f2ab3...,0,120000,21000000000,0xa9059cbb0...,1188463,52481,NaN,NaN,1,2018-12-03 ...,6819732,0x0bfde9e95...
2,0x6df885f19...,670,7,0x29873bd15...,0xbf52f2ab3...,0,120000,21000000000,0xa9059cbb0...,298960,52481,NaN,NaN,1,2018-12-03 ...,6818005,0x41b530f7e...
3,0x6eb3c160e...,672,91,0x29873bd15...,0xbf52f2ab3...,0,120000,21000000000,0xa9059cbb0...,6228779,52481,NaN,NaN,1,2018-12-03 ...,6819820,0xe8b2e7e47...
4,0xcc234f2bd...,99,187,0x010bb230d...,0x89d24a6b4...,0,55371,7000000000,0xa9059cbb0...,7969074,24835,NaN,NaN,0,2018-12-03 ...,6818252,0x543dbce84...


265620

3999

In [17]:
#nd_order_med_gas_from
df.head()
nd_order_med_gas_to_df = df[['to_address', 'gas']]
nd_order_med_gas_to_df = nd_order_med_gas_to_df.groupby('to_address').median()
nd_order_med_gas_to_df = nd_order_med_gas_to_df.reset_index()
nd_order_med_gas_to_df.columns = ['address', '2nd_order_med_gas_to']
fdf = pd.merge(fdf, nd_order_med_gas_to_df, on ='address', how='left')

,hash,nonce,transaction_index,from_address,to_address,value,gas,gas_price,input,receipt_cumulative_gas_used,receipt_gas_used,receipt_contract_address,receipt_root,receipt_status,block_timestamp,block_number,block_hash
0,0xd2ef77134...,673,35,0x29873bd15...,0xbf52f2ab3...,0,120000,21000000000,0xa9059cbb0...,2255311,52481,NaN,NaN,1,2018-12-03 ...,6820184,0x34104d641...
1,0xe216aa7f6...,671,25,0x29873bd15...,0xbf52f2ab3...,0,120000,21000000000,0xa9059cbb0...,1188463,52481,NaN,NaN,1,2018-12-03 ...,6819732,0x0bfde9e95...
2,0x6df885f19...,670,7,0x29873bd15...,0xbf52f2ab3...,0,120000,21000000000,0xa9059cbb0...,298960,52481,NaN,NaN,1,2018-12-03 ...,6818005,0x41b530f7e...
3,0x6eb3c160e...,672,91,0x29873bd15...,0xbf52f2ab3...,0,120000,21000000000,0xa9059cbb0...,6228779,52481,NaN,NaN,1,2018-12-03 ...,6819820,0xe8b2e7e47...
4,0xcc234f2bd...,99,187,0x010bb230d...,0x89d24a6b4...,0,55371,7000000000,0xa9059cbb0...,7969074,24835,NaN,NaN,0,2018-12-03 ...,6818252,0x543dbce84...


In [18]:
#2nd_order_tx_to
len(df)
tx_count_by_address_df = df[['to_address', 'hash']].groupby('to_address').count()
len(tx_count_by_address_df)
tx_count_by_address_df = tx_count_by_address_df.reset_index()
tx_count_by_address_df.columns = ['address', '2nd_order_tx_to']
fdf = pd.merge(fdf, tx_count_by_address_df, on ='address', how='left')
fdf = fdf.fillna(0)
len(fdf)

265620

2522

3999

In [19]:
#fdfa = fdf.loc[(fdf['2nd_order_tx_from'] >= 2) | (fdf['2nd_order_tx_to'] >= 2)]
#fdfa.head(1)
#len(fdfa)
#fdfb = fdf.loc[fdf['2nd_order_tx_to'] >= 1]
#fdfb.head(1)
#len(fdfb)
#fdfa = pd.merge(fdfa, fdfb, on ='address', how='inner')
#len(fdfa)
#fdfa = fdfa.fillna(0)

In [20]:
df2 = pd.read_csv('EthereumClassification/known_address_df.csv')
fdf = pd.merge(fdf, df2, on ='address', how='left')
df2.head()
df2.info()

,Address_ID,address
0,/tokens?l=0x,0x35bfd2cfe...
1,/tokens?l=0x,0xcc70db1ac...
2,/tokens?l=0x,0xcc70db1ac...
3,/tokens?l=0x,0xcb52c65c5...
4,/tokens?l=0x,0x71c7656ec...


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1249 entries, 0 to 1248
Data columns (total 2 columns):
Address_ID    1249 non-null object
address       1249 non-null object
dtypes: object(2)
memory usage: 19.6+ KB


In [23]:
fdf

,address,from_finex_tx,from_finex_vol,vol_per_tx,is_smart_contract,2nd_order_med_gas_from,2nd_order_tx_from,2nd_order_vol_from,2nd_order_vol_per_tx_from,2nd_order_smart_contract_%,2nd_order_med_gas_to,2nd_order_tx_to,Address_ID
0,0x2b1405e72...,2,3.769213e+17,1.884606e+17,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000,0.0,0.0,NaN
1,0x8b7bffc6f...,1,8.000000e+19,8.000000e+19,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000,0.0,0.0,NaN
2,0x28571a885...,1,2.628879e+18,2.628879e+18,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000,0.0,0.0,NaN
3,0x95a254f6c...,1,1.297280e+18,1.297280e+18,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000,0.0,0.0,NaN
4,0xb024c0664...,3,1.316550e+19,4.388500e+18,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000,0.0,0.0,NaN
5,0x65eca6289...,4,4.404717e+20,1.101179e+20,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000,0.0,0.0,NaN
6,0x99045f200...,1,5.003113e+17,5.003113e+17,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000,0.0,0.0,NaN
7,0x244596650...,1,5.670466e+17,5.670466e+17,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000,0.0,0.0,NaN
8,0xcf51a1b2a...,3,3.499460e+19,1.166487e+19,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000,0.0,0.0,NaN
9,0xb8ab6335f...,1,1.144375e+18,1.144375e+18,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000,0.0,0.0,NaN


In [22]:
fdf.to_csv('fdf.csv', index=False)